In [91]:
import pandas as pd
import datetime

In [92]:
path = "Major_Crime_Indicators_Open_Data.csv"

data_df = pd.read_csv(path)
data_df.head()

,X,Y,OBJECTID,EVENT_UNIQUE_ID,REPORT_DATE,OCC_DATE,REPORT_YEAR,REPORT_MONTH,REPORT_DAY,REPORT_DOY,...,UCR_CODE,UCR_EXT,OFFENCE,MCI_CATEGORY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,LONG_WGS84,LAT_WGS84
0,-79.425896,43.757346,1,GO-20141262074,2014/01/01 05:00:00+00,1998/06/01 04:00:00+00,2014,January,1,1,...,1480,110,Administering Noxious Thing,Assault,38,Lansing-Westgate,38,Lansing-Westgate (38),-79.425896,43.757346
1,-79.350229,43.646293,2,GO-20141260701,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,...,2120,200,B&E,Break and Enter,70,South Riverdale,70,South Riverdale (70),-79.350229,43.646293
2,-79.376497,43.666423,3,GO-20141260889,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,...,1430,100,Assault,Assault,74,North St.James Town,74,North St.James Town (74),-79.376497,43.666423
3,-85.488744,0.000000,4,GO-20141260973,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,...,2130,210,Theft Over,Theft Over,NSA,NSA,NSA,NSA,-85.488744,0.000000
4,-79.344839,43.678946,5,GO-20141261050,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,...,1430,100,Assault,Assault,69,Blake-Jones,66,Danforth (66),-79.344839,43.678946


In [93]:
final_df = data_df[['X', 'Y', 'OBJECTID', 'EVENT_UNIQUE_ID', 'REPORT_DATE', 'OCC_DATE',
        'OCC_YEAR', 'OCC_MONTH','OCC_HOUR', 'DIVISION', 'LOCATION_TYPE', 'PREMISES_TYPE', 'OFFENCE', 'MCI_CATEGORY', 'HOOD_158', 'NEIGHBOURHOOD_158',
       'HOOD_140', 'NEIGHBOURHOOD_140']]
final_df = final_df[final_df['OCC_YEAR']>2015]

In [94]:
# Convert OCC_YEAR and OCC_MONTH columns to integers
final_df['OCC_YEAR'] = final_df['OCC_YEAR'].astype(int)
final_df['OCC_MONTH'] = final_df['OCC_MONTH'].apply(lambda x: pd.to_datetime(x, format='%B').month)

In [95]:

crime_types = ['Assault', 'Auto Theft', 'Break and Enter', 'Robbery', 'Theft Over']
# Create a list to store the aggregated crime data
aggregated_data = []
for year in range(2016, 2023):
    year_df = final_df[final_df['OCC_YEAR'] == year]

    # Loop through each month
    for month in range(1, 13):  # Loop through months 1 to 12
        month_df = year_df[year_df['OCC_MONTH'] == month]

        # Initialize a dictionary to store crime counts
        crime_counts = {crime_type: 0 for crime_type in crime_types}

        # Loop through the filtered DataFrame and count crimes
        for index, row in month_df.iterrows():
            crime_type = row['MCI_CATEGORY']  # Adjust column name as needed
            if crime_type in crime_counts:
                crime_counts[crime_type] += 1

         # Append the aggregated data to the list
        aggregated_data.append({
            'Year': year,
            'Month': month,
            **crime_counts
        })




In [96]:
# Create a DataFrame from the aggregated data
aggregated_df = pd.DataFrame(aggregated_data)


In [97]:
aggregated_df

,Year,Month,Assault,Auto Theft,Break and Enter,Robbery,Theft Over
0,2016,1,1543,271,546,270,63
1,2016,2,1384,222,525,304,85
2,2016,3,1556,281,504,274,82
3,2016,4,1574,230,539,323,82
4,2016,5,1696,254,501,318,100
...,...,...,...,...,...,...,...
79,2022,8,1849,755,500,258,125
80,2022,9,1798,852,514,265,116
81,2022,10,1804,955,502,251,120
82,2022,11,1662,945,523,276,132


In [98]:
aggregated_df.columns = aggregated_df.columns.str.replace(' ', '_')
aggregated_df["Id"] = aggregated_df.index.values

In [99]:
aggregated_df.head()

,Year,Month,Assault,Auto_Theft,Break_and_Enter,Robbery,Theft_Over,Id
0,2016,1,1543,271,546,270,63,0
1,2016,2,1384,222,525,304,85,1
2,2016,3,1556,281,504,274,82,2
3,2016,4,1574,230,539,323,82,3
4,2016,5,1696,254,501,318,100,4


In [100]:
# Save the DataFrame to a CSV file
aggregated_df.to_csv('crime_counts.csv', index=False)

In [103]:
import csv
import json
input_file = 'crime_counts.csv'
output_file = 'crime_counts.json'

csv_records = []
with open(input_file, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        csv_records.append(row)

# Write JSON with commas between records
with open(output_file, 'w') as json_file:
    json_file.write('[')
    for index, record in enumerate(csv_records):
        json.dump(record, json_file, indent=2)
        if index < len(csv_records) - 1:
            json_file.write(',')
    json_file.write(']')